# Env

In [1]:
from pyspark.sql import SparkSession
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from src.processing.process_data import (
    format_date, format_bdi, format_spec,
    to_numeric, remove_duplicates, remove_outliers,
    prep_ml
)
from sklearn.preprocessing import StandardScaler
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, to_date, regexp_replace

# Dataframe

In [2]:
spark = SparkSession\
    .builder\
    .master('local[*]')\
    .appName('load')\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/09 15:52:17 WARN Utils: Your hostname, Giordano, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/09 15:52:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 15:52:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark\
    .read\
    .parquet('../../datalake/Bovespa')\
    .filter('year >= 1994 AND month >= 7')

# Analysis

In [ ]:
numeric_columns = [
    'Opening Price', 'Max. Price', 'Min. Price',
    'Mean Price', 'Last Trade Price', 'Best Purshase Order Price',
    'Best Purshase Sale Price', 'Numbor Of Trades', 'Number Of Traded Stocks',
    'Volume Of Traded Stocks'
]

df = remove_duplicates(df)
df = to_numeric(df, numeric_columns)
df = remove_outliers(df, numeric_columns)
df = format_date(df)
df = format_bdi(df)
df = format_spec(df)

In [5]:
df\
    .coalesce(1)\
    .write\
    .csv(
        '../../datalake/serving/bovespa_plano_real.csv',
        mode='overwrite',
        header=True,
        sep=';'
    )

# Machine Learning

In [6]:
folder = '../../datalake/serving/bovespa_plano_real.csv'
file_name = [f for f in os.listdir(folder) if f.endswith('.csv')][0]
df = spark.read.csv(
        '../../datalake/serving/bovespa_plano_real.csv',
        header=True,
        sep=';'
    )
df_prep = prep_ml(df)

In [7]:
train_df, test_df = df_prep.randomSplit([0.8, 0.2], seed=42)

train_df.write.mode("overwrite").parquet("../../datalake/serving/train_bovespa_plano_real.parquet")
test_df.write.mode("overwrite").parquet("../../datalake/serving/test_bovespa_plano_real.parquet")

25/11/09 15:56:47 WARN DAGScheduler: Broadcasting large task binary with size 1058.6 KiB
25/11/09 15:57:03 WARN DAGScheduler: Broadcasting large task binary with size 1058.6 KiB
